In [7]:
import Bio
from Bio.PDB import *
import sys
import re
#import matplotlib.pyplot as plt
#from matplotlib import cm
import math
import numpy as np
import os

import pandas
import multiprocessing

import shutil


#This is for removing Bio.python warnings
import warnings
from Bio import BiopythonWarning
warnings.simplefilter('ignore', BiopythonWarning)


In [8]:
def calc_residue_dist(residue1, residue2, outfile, atom1_used, atom2_used) :

    for atom1 in residue1:
        for atom2 in residue2:

            distance=atom1-atom2    

            if distance>20:#If the residues is to far from eachother there is no need to calculate the distance for the rest of the atoms
                return
            if distance >12:#Go to the next atom
                continue
                
            if 'H' not in atom1.get_id() and 'H' not in atom2.get_id(): # DUBBELKOLLA ATT DET INTE FINNS H I ANDRA!!!!
            #if atom1.get_id() != 'H' and atom2.get_id() != 'H':    
                
                a1_serial=str(atom1.serial_number)
                a2_serial=str(atom1.serial_number)
                if a1_serial not in atom1_used:
                    atom1_used.append(a1_serial)
                    x=str(atom1.get_coord()[0])
                    y=str(atom1.get_coord()[1])
                    z=str(atom1.get_coord()[2])
                    if 'C' in atom1.get_name():# and
                        outfile.write('0'+' '+'1'+' '+x+' '+y+' '+z+'\n')
                    elif 'O' in atom1.get_name():
                        outfile.write('0'+' '+'2'+' '+x+' '+y+' '+z+'\n')
                    elif 'N' in atom1.get_name():
                        outfile.write('0'+' '+'3'+' '+x+' '+y+' '+z+'\n')
                    elif 'S' in atom1.get_name():
                        outfile.write('0'+' '+'4'+' '+x+' '+y+' '+z+'\n')
                if a2_serial not in atom2_used:
                    atom2_used.append(a2_serial)
                    x=str(atom2.get_coord()[0])
                    y=str(atom2.get_coord()[1])
                    z=str(atom2.get_coord()[2])
                    if 'C' in atom1.get_name():# and
                        outfile.write('1'+' '+'1'+' '+x+' '+y+' '+z+'\n')
                    elif 'O' in atom1.get_name():
                        outfile.write('1'+' '+'2'+' '+x+' '+y+' '+z+'\n')
                    elif 'N' in atom1.get_name():
                        outfile.write('1'+' '+'3'+' '+x+' '+y+' '+z+'\n')
                    elif 'S' in atom1.get_name():
                        outfile.write('1'+' '+'4'+' '+x+' '+y+' '+z+'\n')
                
    return




def calc_dist_matrix(chain_one, chain_two,outfile) :
    
    atom1_used=[]
    atom2_used=[]
    for row, residue_one in enumerate(chain_one) :
        for col, residue_two in enumerate(chain_two) :

            calc_residue_dist(residue_one, residue_two, outfile,atom1_used,atom2_used)
                           

            
    return

In [9]:
def main():
    
    #Dir path
    #args = sys.argv[1:]
    pdb_dir= '/home/joakim/Downloads/models' #str(args[0]) #'D:\Downloads\CnM-dataset\models'
    cwd = os.getcwd()
    
    while 'true':
        input1=raw_input("Save or load ")
        if input1.lower()=='-r':
            args='-r'
            break
        elif input1.lower()=='-s':
            args='-s'
            break
        else:
            break
    
    
    
    
    if not '-r' in args:
    
        pdb_list=[]
        for file in os.listdir(pdb_dir):
            if file.endswith(".pdb"):
                pdb_list.append([pdb_dir,file])
      
        valuefile='/home/joakim/Downloads/CnM.featuresNPqDNZ'#'D:\Downloads\CnM.featuresNPqDNZ'
        value_df=pandas.read_csv(valuefile,delim_whitespace=1)

        #Just a check to se if the data_set dir exist, in this case it removes it and all the files in it and make a new one
        if os.path.exists(cwd + "/pts_dir"):
            shutil.rmtree(cwd + "/pts_dir")
            os.makedirs(cwd + "/pts_dir")
        else:
            os.makedirs(cwd + "/pts_dir")
        


        
        score=[]
        dist_matrix=[]
        for counter, filepath_pdb in enumerate(pdb_list):
            
            working_file=open(cwd + '/pts_dir/working_file.txt', 'a')
            working_file.write(pdb_list[counter][1][:-11]+'\n') #Writes what pdb have been worked on in working_file.txt
            working_file.close()
            
            print(counter)
            print(pdb_list[counter][1])
            filename_pdb=os.path.join(pdb_list[counter][0],pdb_list[counter][1])

            try: 
                PDBobj = PDBParser()
                structure = PDBobj.get_structure(filename_pdb, filename_pdb)
                model = structure[0]
            except IOError: 
                print('IO Error', filename_pdb)      
                while 'true':
                    input1=raw_input("Error parsing PDB file! Continue(y/n)...")
                    if input1.lower()=='y':
                        continue
                    elif input1.lower()=='n':
                        sys.exit()

           
            score.append(value_df.loc[((value_df['#'] == pdb_list[counter][1][:-11]),'CPscore')].values[0])



            chain_used=[]
            outfile=open(cwd + "/pts_dir/"+pdb_list[counter][1][:-11] + '.pts', 'w')
            for chain1 in model:
                for chain2 in model:
                    if chain1!=chain2 and chain2 not in chain_used:
                        chain_used.append(chain1) 
                        calc_dist_matrix(chain1, chain2, outfile)
            outfile.close()
    



    if '-s' in args and '-r' in args:
        print('Do not use save(-s) and read(-r) at the same time')
        sys.exit()
    elif '-s' in args:
        print('-Saving files...')
        np.savez_compressed('score', score)
        #np.savez_compressed('dist_matrix', dist_matrix)
        print('-Files saved as "dist_matrix.npz","score.npz"')

    elif '-r' in args:
        print('-Loading files...')
        score = np.load('score.npz')
        #dist_matrix = np.load('dist_matrix.npz')
        
        for key,array in score.items():
            score=score[key]
        #for key,array in dist_matrix.items():
            #dist_matrix=dist_matrix[key]
            
        print('-Files loaded')
    
    
    
    print(score[1])
    print(score.shape)
 
    
if __name__ == '__main__':
    main()

Save or load -s
0
D1CGI-a2c-merged.pdb
1
D1I2M-a4c-merged.pdb
2
D1A2K-a0a-merged.pdb
3
D1HIA-a2b-merged.pdb
4
D1Z5Y-a87a-merged.pdb
-Saving files...
-Files saved as "dist_matrix.npz","score.npz"
0.027


AttributeError: 'list' object has no attribute 'shape'